# Quiz 7 

### Question 3
The figure below shows a Recurrent Neural Network (RNN) with one input unit x, one logistic hidden unit h, and one linear output unit $y$

The RNN is unrolled in time for T=0,1, and 2.

![RNN1](misc/rnn1.png)

The network parameters are: $W_{xh} = 0.5, W_{hh} = -1.0, W_{hy} = -0.7$, $h_{bias}=-0.1$ and $y_{bias}=0.0$. Remember, $\sigma(k) = \frac{1}{1 + e^{-k}}$.

The inputs at various time steps are the following:

$T$ | $x$ (input)
--- | ---
0 | 9
1 | 4
2 | -2


What is the value of the output y at $T=1$?

**How to forward propagate in a recurrent neural network?**

We can think of the given neural net as a growing rnn where, after each time-step, an input is produced, the output of the net at time $t$ does not affect the hidden units in the forward progation. It then follows that in order to find the output at time 2 of this RNN, we only forward propagate twice with the same hidden units (using the logistic function), and compute the output with the linear activation function

In [7]:
from math import exp
sigma = lambda k: 1 / (1 + exp(-k))
X = [9, 4]
Wxh, Whh, Why = 0.5, -0.1, -0.7
hbias = -0.1

# hidden from time 0:
#    Input to hidden state
o1 = sigma(X[0] * Wxh + hbias)
# hidden from time 1:
#    output from time 0 and input from time 1
o2 = sigma((o1 * Whh) + (X[1] * Wxh) + hbias)
# output from time 1
output = o2 * Why
print(output)

-0.6008075567320005


### Question 4
Consider the RNN architecture above.

The network parameters are:

Params | Values
--- | ---
$W_{xh}$ | -0.1
$W_{hh}$ | 0.5
$W_{hy}$ | 0.25
$h_{bias}$ | 0.4
$y_{bias}$ | 0.0

And the inputs are

$T$ | $x$ (input)
--- | ---
0 | 18
1 | 9
2 | -8

In [6]:
X = [18, 9, -8]
Wxh, Whh, Why = -0.1, 0.5, 0.25
hbias = 0.4

sigma(X[0] * Wxh + hbias)

0.19781611144141825